# Fix Classes=Categories
version: 3

author: nuno costa

In [2]:
from annotate_v5 import *
import platform 
import numpy as np
import pandas as pd
from IPython.display import Image, display
import copy
import os
from shutil import copyfile
import matplotlib.pyplot as plt
from matplotlib.image import imread
from matplotlib.patches import Rectangle
import random

#Define root dir dependent on OS
rdir='D:/external_datasets/' 
if str(platform.platform()).find('linux')>-1: rdir='/mnt/d/external_datasets/' 
print('OS: {}'.format(platform.platform()))
print('root dir: {}'.format(rdir))

OS: Windows-10-10.0.20241-SP0
root dir: D:/external_datasets/


In [3]:
mlabjson =  json.load(open(rdir+'mlab_cocotao.json'))

In [4]:
for k in mlabjson:
    print(k, len(mlabjson[k]))

info 5
licenses 9
categories 1310
videos 1488
images 177936
tracks 8132
segment_info 0
annotations 1954769
datasets 2


## 1. Import ids
#### #NOTE: work with ids and index so you can use numpy for faster operations
#### #WARNING don't use try: except: pass when importing - go back to mergedatasets and find the BUG. 

In [4]:
# datasets name and id
dset_l=[]
dset_l_id=[]
for d in mlabjson['datasets']:
    dset_l.append(d['name'])
    dset_l_id.append(d['id'])
print(dset_l, dset_l_id)

['COCO', 'TAO'] [1, 2]


In [5]:
# categories name and id
cat_l=[]
cat_l_id=[]
cat_l_dset=[]
for c in mlabjson['categories']:
    cat_l.append(c['name'])
    cat_l_id.append(c['id'])
    cat_l_dset.append(dset_l[c['dataset']-1]) # dset_l index is same as id-1
#print(cat_l_id)

In [6]:
# images filepath and id
img_l=[]
img_l_id=[]
for c in mlabjson['images']:
    img_l.append(c['file_name'])
    img_l_id.append(c['id'])

In [7]:
# annotations category_id, image_id, bbox, and dataset
ann_catid=[]
ann_imgid=[]
ann_bbox=[]
ann_dset=[]
for an in tqdm(mlabjson['annotations']):
    ann_catid.append(an['category_id'])
    ann_imgid.append(an['image_id'])
    ann_bbox.append(an['bbox'])
    ann_dset.append(an['dataset'])

100%|█████████████████████████████████████████████████████████████████| 1954769/1954769 [00:01<00:00, 1201465.67it/s]


## 2. Find duplicates cat_ids
Duplicates example
categories= [{name:cow, id:1, dataset:1},...,{name:cow, id:200, dataset:2},...,{name:cow, id:101, dataset:3}]}

In [8]:
#duplicates #TODO: SORT alphabetically
duplicates_l=[]
duplicates_l_catid=[]
duplicates_l_catdset=[]
duplicate_method="equal_names"
if duplicate_method=="equal_names": #FIX REPETITIONS
    duplicates_l=list(set([x for x in cat_l if cat_l.count(x) > 1])) # duplicates l 
    for duplicate in tqdm(duplicates_l):
        idx_mask = [name == duplicate for name in cat_l] #mask of index of duplicate
        catids = np.array(cat_l_id)[idx_mask] #duplicate catids
        catdsets = np.array(cat_l_dset)[idx_mask] #duplicate catdset
        duplicates_l_catid.append(catids.tolist())
        duplicates_l_catdset.append(catdsets.tolist())
if duplicate_method=="similar_names": #FIX SIMILAR NAMES
    #WARNING: fix equal_names first - if not, it will not be fixed in this case
    import difflib
    for cat in cat_l: 
        match_l=[]
        match_l_temp=[c for c in cat_l if (c).find(cat)>-1] #only substring inside string - catagory with big name is not found
        if not len(match_l_temp)>1: continue #more than one
        #refine search #ADD or REMOVE refine options
        match_l.append(cat) #make category to be the first item
        match_l.extend([c for c in match_l_temp if ((c+' ').find(cat)>-1 or (c+' ').find(cat)>-1 or (' '+c).find(cat)>-1 or (c+'_').find(cat)>-1 or ('_'+c).find(cat)>-1 )]) #mantain optins: c+'_';'_'+c 
        match_l.extend(difflib.get_close_matches(cat, match_l_temp, n=5, cutoff=0.9)) #get similar words
        #remove equal names
        match_l=list(dict.fromkeys(match_l))#match_l=list(set(match_l))
        if not len(match_l)>1: continue #more than one
        #add to duplicates
        duplicates_l.append(match_l)
    for duplicates in tqdm(duplicates_l):
        catids=[]
        catdsets=[]
        for duplicate in duplicates:
            idx=cat_l.index(duplicate) #first duplicate ()
            catids.append(cat_l_id[idx]) #duplicate catids
            catdsets.append(cat_l_dset[idx]) #duplicate cat dsets
        duplicates_l_catid.append(catids)
        duplicates_l_catdset.append(catdsets)
if duplicate_method=="all_names": #Do for all category names, even with equal 
    duplicates_l=cat_l #NORMALIZE: same list format ?? not needed
    duplicates_l_catid=[[id] for id in cat_l_id]
    duplicates_l_catdset=[[dset] for dset in cat_l_dset]
    

print(duplicates_l)
print(duplicates_l_catid)
print(duplicates_l_catdset)
print(len(duplicates_l))
print(len(duplicates_l_catid))
print(len(duplicates_l_catdset))

100%|██████████████████████████████████████████████████████████████████████████████| 49/49 [00:00<00:00, 1042.57it/s]

['fork', 'chair', 'clock', 'bird', 'toaster', 'horse', 'umbrella', 'bench', 'skateboard', 'zebra', 'elephant', 'sheep', 'carrot', 'handbag', 'bottle', 'toothbrush', 'pizza', 'knife', 'boat', 'dog', 'apple', 'bed', 'vase', 'scissors', 'cake', 'book', 'bear', 'toilet', 'giraffe', 'suitcase', 'kite', 'spoon', 'bicycle', 'broccoli', 'surfboard', 'motorcycle', 'frisbee', 'backpack', 'cup', 'sink', 'sandwich', 'snowboard', 'cow', 'banana', 'truck', 'cat', 'airplane', 'refrigerator', 'bowl']
[[43, 555], [57, 317], [75, 356], [15, 179], [71, 1195], [18, 659], [26, 1235], [14, 171], [37, 1060], [23, 1309], [21, 509], [19, 1041], [52, 301], [27, 115], [40, 213], [80, 1202], [54, 909], [44, 705], [9, 198], [17, 462], [48, 93], [60, 159], [76, 1242], [77, 1017], [56, 265], [74, 206], [22, 158], [62, 1197], [24, 582], [29, 116], [34, 701], [45, 1098], [2, 175], [51, 234], [38, 1137], [4, 794], [30, 560], [25, 114], [42, 427], [72, 1059], [49, 1006], [32, 1073], [20, 161], [47, 125], [8, 1224], [16,

In [9]:
# get annotations duplicates
ann_catid_np=np.array(ann_catid)
ann_imgid_np=np.array(ann_imgid)
ann_bbox_np=np.array(ann_bbox)
ann_dset_np=np.array(ann_dset)
duplicates_l_imgid=[]
duplicates_l_bbox=[]
duplicates_l_dset=[]
for catids in tqdm(duplicates_l_catid):
    l_imgid=[]
    l_bbox=[]
    l_dset=[]
    for catid in catids:
        ann_idx = np.where(ann_catid_np==catid)[0].tolist() #annotation index of ids
        l_imgid.append(ann_imgid_np[ann_idx].tolist())
        l_bbox.append(ann_bbox_np[ann_idx].tolist())
        l_dset.append(ann_dset_np[ann_idx].tolist())
    duplicates_l_imgid.append(l_imgid)
    duplicates_l_bbox.append(l_bbox)
    duplicates_l_dset.append(l_dset)

100%|████████████████████████████████████████████████████████████████████████████████| 49/49 [00:00<00:00, 95.52it/s]


## 3. Classes|categories to fix 

In [15]:
classtofix_l=[]
classtofix_l_catid=[]
method="all"
if method=="all": #fix all duplicates
    classtofix_l=duplicates_l
    classtofix_l_catid=duplicates_l_catid
if method=="view_images": # view images and select class to fix (INCEPTION =>>>Pufff!!!!)
    img_l_id_np=np.array(img_l_id)      
    for i, duplicate in enumerate(tqdm(duplicates_l)): #run for each duplicate
        print(duplicate+'...') #class
        display_imgs(rdir, i, dset_l, duplicates_l_catid, duplicates_l_dset, duplicates_l_imgid, img_l, img_l_id, imgidx=0)
        classtofix_l, classtofix_l_catid=save_classtofix(i, classtofix_l, classtofix_l_catid, duplicates_l, duplicates_l_catid, duplicates_l_dset, duplicates_l_imgid, img_l, img_l_id, imgidx=0)
if method=="save_images": # save images to folder for manual check
    datadir="duplicates"
    folder=duplicate_method+'/'
    showimage=False #show images
    startidx=0 # start index of image to save from each dataset
    imgnr=10 # total number of images to save from each dataset
    imgstep='random' # step between images: int | 'random' - int steps between images; 'rand' gets random list
    path=os.path.join(rdir,datadir,folder)
    assure_path_exists(path)
    #TODO
    #save excel # TODO Send duplicates_l, duplicates_L_catid, duplicates_l_catdset: user should make column  classtofix_l and classtofix_L_catid - user should create  
    df=pd.DataFrame({'duplicates_l': duplicates_l,'duplicates_l_catid': duplicates_l_catid, 'duplicates_l_catdset': duplicates_l_catdset, 'classtofix_l': np.nan, 'classtofix_l_catid':np.nan, 'rules':np.nan })
    df.loc[0, 'rules']="To fix classes: 1) You need to fill the column classtofix_l and/or classtofix_l_catid with the information from the respective duplicate columns; 2) When copy/pasting or changing, make sure the same structure maintains:  ['car', 'carrot'], [3, 52], beware of the spaces ['car', '  and always maintain the first class in the list;  3) You have 3 possibilities of filling the columns : 1-the 2 columns empty, meaning the row will not be used for classtofix; 2-only one column empty, e.g. fill the classtotix_l row with the class labels from duplicates_l, then during the importing the classtofix_l_catid is filled, and vice-versa; 3-If you want to change the name of the first class in the list,e.g ['car', 'carrot'] for ['automobile', 'carrot'] you need to provide the ids to classtofix_l_catid."
    excelpath=path+duplicate_method+"_classtofix_report.xlsx"
    df['annotations_missing'] = np.empty((len(df), 0)).tolist()
    df['images_missing'] = np.empty((len(df), 0)).tolist()
    #save image for each duplicate
    for i, duplicate in enumerate(tqdm(duplicates_l)): #run for each duplicate category
        minclass=duplicate
        if isinstance(minclass, list): minclass=minclass[0]
        print('\n>> '+minclass+'...') #class
        classpath=os.path.join(path, minclass) # folder for images of this class
        classpath=parse_path(classpath)+'/' #make it a folder
        assure_path_exists(classpath)
        df=save_imgs(df, rdir, classpath, i, dset_l, duplicates_l, duplicates_l_catid, duplicates_l_bbox, duplicates_l_dset,
              duplicates_l_imgid, img_l, img_l_id, startidx=startidx, imgnr=imgnr, imgstep=imgstep, showimage=showimage)
    df.to_excel(excelpath, index=False)   
if method=="view_duplicate":
    i=-1 #duplicate_l index : last category
    ii=1 #dset_l index : TAO
    step=10 #step images
    dpi=80
    dataset=dset_l[ii]
    category=duplicates_l[i]
    view_duplicate(rdir, i, ii, dataset, category, duplicates_l_imgid, duplicates_l_bbox, img_l, img_l_id, step=step,
                   dpi=dpi)

  0%|                                                                                         | 0/49 [00:00<?, ?it/s]


>> fork...
COCO



>>> Finding COCO fork...:   0%|                                                             | 0/5694 [00:00<?, ?it/s]

[Errno 2] No such file or directory: 'D:/external_datasets/COCO/2017/images/val2017/000000084664.jpg'




>>> Finding TAO fork...:   0%|                                                               | 0/167 [00:00<?, ?it/s]

...



>>> Finding TAO bowl...:   0%|                                                       | 1/505 [00:00<00:30, 16.39it/s]



>>> Finding TAO bowl...:   0%|▏                                                      | 2/505 [00:00<00:14, 33.88it/s]

>>> Finding TAO bowl...:   0%|                                                               | 0/505 [00:00<?, ?it/s]

[Errno 2] No such file or directory: 'D:/external_datasets/TAO/TAO_DIR/frames/val/YFCC100M/v_b4fbb0da6968177ee3d7f31aa3f6648/frame1201.jpg'
[Errno 2] No such file or directory: 'D:/external_datasets/TAO/TAO_DIR/frames/val/Charades/OEIR9/frame1016.jpg'


>>> Finding TAO bowl...:   1%|▎                                                      | 3/505 [00:00<00:10, 50.01it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 49/49 [02:31<00:00,  3.08s/it]

[Errno 2] No such file or directory: 'D:/external_datasets/TAO/TAO_DIR/frames/val/YFCC100M/v_460a1314da9bd32cbd14a2b3414f30/frame0649.jpeg'


In [11]:
#WARNING: CHECK EXCEL FIRST
if method=="save_images": # reading classtofix from excel
    classtofix_l=[]
    classtofix_l_catid=[]
    df=pd.read_excel(excelpath)
    classtofix_df=df.loc[:,'classtofix_l']
    classtofix_df_catid=df.loc[:,'classtofix_l_catid']
    display(df)
    # PARSE COLUMNS TO FIX
    classtofix_l=classtofix_df.tolist()
    classtofix_l_catid=classtofix_df_catid.tolist()
    print(classtofix_l_catid)
    #convert strings to lists
    for icl, cl in enumerate(classtofix_l): 
        if isinstance(classtofix_l[icl], str): classtofix_l[icl]=convert_unicode(classtofix_l[icl], method='liststr')
        if isinstance(classtofix_l_catid[icl], str): classtofix_l_catid[icl]=convert_unicode(classtofix_l_catid[icl], method='listnum')
    print(classtofix_l_catid)
    #parse the columns based on the rules    
    for ic, classes in enumerate(classtofix_df):
        #1. Two columns empty - do nothing, maintain
        if  pd.isnull(classtofix_df.iloc[ic]) and pd.isnull(classtofix_df_catid.iloc[ic]): continue
        #2. if only classtofix_l_catid empty - get 
        if not pd.isnull(classtofix_df.iloc[ic]) and pd.isnull(classtofix_df_catid.iloc[ic]):
            classes=convert_unicode(classes, method='liststr')
            cids=[]
            for c in classes:
                cidx=duplicates_l[ic].index(c)
                cid=duplicates_l_catid[ic][cidx]
                cids.append(cid)
            classtofix_l_catid[ic]=cids
            print(classtofix_l_catid)
        #2. if only classtofix_l empty - get
        if pd.isnull(classtofix_df.iloc[ic]) and not pd.isnull(classtofix_df_catid.iloc[ic]): 
            classes=[]
            cids=classtofix_l_catid[ic]
            for c in cids:
                cidx=duplicates_l_catid[ic].index(c)
                clas=duplicates_l[ic][cidx]
                classes.append(clas)
            classtofix_l[ic]=classes
        #3. if both columns not empty - do nothing
        if not pd.isnull(classtofix_df.iloc[ic]) and not pd.isnull(classtofix_df_catid.iloc[ic]): continue

In [12]:
print('>> Make sure everything is correct: 1.Drop NaN if exist, but make sure the index is the same for the two! \n')
fixempty=True
if fixempty:
    classtofix_l=[x for x in classtofix_l if str(x) != 'nan' and str(x) !='[]']
    classtofix_l_catid=[x for x in classtofix_l_catid if str(x) != 'nan' and str(x) !='[]']
print(classtofix_l)
print(classtofix_l_catid)

>> Make sure everything is correct: 1.Drop NaN if exist, but make sure the index is the same for the two! 

['fork', 'chair', 'clock', 'bird', 'toaster', 'horse', 'umbrella', 'bench', 'skateboard', 'zebra', 'elephant', 'sheep', 'carrot', 'handbag', 'bottle', 'toothbrush', 'pizza', 'knife', 'boat', 'dog', 'apple', 'bed', 'vase', 'scissors', 'cake', 'book', 'bear', 'toilet', 'giraffe', 'suitcase', 'kite', 'spoon', 'bicycle', 'broccoli', 'surfboard', 'motorcycle', 'frisbee', 'backpack', 'cup', 'sink', 'sandwich', 'snowboard', 'cow', 'banana', 'truck', 'cat', 'airplane', 'refrigerator', 'bowl']
[[43, 555], [57, 317], [75, 356], [15, 179], [71, 1195], [18, 659], [26, 1235], [14, 171], [37, 1060], [23, 1309], [21, 509], [19, 1041], [52, 301], [27, 115], [40, 213], [80, 1202], [54, 909], [44, 705], [9, 198], [17, 462], [48, 93], [60, 159], [76, 1242], [77, 1017], [56, 265], [74, 206], [22, 158], [62, 1197], [24, 582], [29, 116], [34, 701], [45, 1098], [2, 175], [51, 234], [38, 1137], [4, 794]

## 4. Fix classes

In [14]:
# slow # newjson=copy.deepcopy(mlabjson) #do deepcopy to compare
# fast
newjson={'categories':[],'annotations':[] }
newjson['categories']=copy.copy(mlabjson['categories'])
newjson['annotations']=copy.copy(mlabjson['annotations'])

In [13]:
classtofix_l_catidx=[[cat_l_id.index(id) for id in id_l] for id_l in classtofix_l_catid]
#print(classtofix_l_catidx) # they should be less one, becacuse it is ordered
print(len(classtofix_l_catidx))

49


#### Change mlabjson['categories']: [{name: , id: }]  
=>  use min cat id and remove the other categories (!!!Without ordering again the category id!!!)

In [16]:
removeidx_l=[]
minidx=0 # get first cat: minimum in equal names and the category used to search in similar_names
for i,id_l in enumerate(tqdm(classtofix_l_catid)): #for each classtofix
    minid=id_l[minidx] # #category id 
    catidx=classtofix_l_catidx[i][minidx]# get cat index of min catid
    if isinstance(classtomix_l[i], list): newjson['categories'][firstcatidx]['name']=classtomix_l[i][minidx] #change name of first id 
    else: newjson['categories'][catidx]['name']=classtofix_l[i] #change name of min id (if changed)
    assert newjson['categories'][catidx]['id']==id_l[minidx] #assert id - it should be the same
    otheridx_l=copy.copy(classtofix_l_catidx[i]) #the idx to remove
    otheridx_l.remove(catidx)
    removeidx_l.extend(otheridx_l) #remove index
removeidx_l=list(dict.fromkeys(removeidx_l)) # no duplicates

0it [00:00, ?it/s]


In [17]:
#REMOVE - Newjson will be changed
removeitem_l=[newjson['categories'][removeidx] for removeidx in removeidx_l] #items to remove
for removeitem in removeitem_l: newjson['categories'].remove(removeitem) 

In [18]:
if not True: print(True)
else: print(False)

False


In [19]:
#TEST
print(mlabjson['categories'][13])
print(newjson['categories'][13])
print(mlabjson['categories'][170])
print(newjson['categories'][170])

{'supercategory': 'outdoor', 'id': 14, 'name': 'bench', 'dataset': 1}
{'supercategory': 'outdoor', 'id': 14, 'name': 'bench', 'dataset': 1}
{'frequency': 'f', 'id': 171, 'synset': 'bench.n.01', 'image_count': 93, 'instance_count': 160, 'synonyms': ['bench'], 'def': 'a long seat for more than one person', 'name': 'bench', 'dataset': 2}
{'frequency': 'f', 'id': 171, 'synset': 'bench.n.01', 'image_count': 93, 'instance_count': 160, 'synonyms': ['bench'], 'def': 'a long seat for more than one person', 'name': 'bench', 'dataset': 2}


#### change mlabjson['annotations']: [{category_id: , }] 
=> remove_catid_l to annidx_l

In [18]:
ann_catid_np=np.array(ann_catid)
classtofix_l_ann_catidx=[[np.where(ann_catid_np==id)[0].tolist()  for id in id_l] for id_l in classtofix_l_catid]
print(len(classtofix_l_ann_catidx))
print(classtofix_l_catidx[0][0])
print(len(classtofix_l_ann_catidx[0][0]))
print((classtofix_l_ann_catidx[0][0][0]))#to maintain same id
print((classtofix_l_ann_catidx[0][1][0]))#to change same id

49
17
6860
681
1172463


In [19]:
for i,id_l in enumerate(classtofix_l_catid): #for each classtofix
    minidx=0 # get first catid min
    minid=id_l[minidx]
    ann_catidx= classtofix_l_ann_catidx[i][minidx]# get annotation cat index of min catid
    ann_otheridx_l=copy.copy(classtofix_l_ann_catidx[i])
    ann_otheridx_l.remove(ann_catidx) #the idx to change
    for ann_otheridx in ann_otheridx_l[0]: newjson['annotations'][ann_otheridx]['category_id']=minid
        

In [20]:
#TEST
print(mlabjson['annotations'][8875])
print(newjson['annotations'][8875])
print(mlabjson['annotations'][1174300])
print(newjson['annotations'][1174300])

{'segmentation': [[20.95, 105.11, 20.95, 87.58, 8.43, 80.69, 8.43, 75.06, 38.48, 74.43, 40.35, 66.92, 32.22, 63.79, 39.1, 35.62, 135.51, 37.5, 130.5, 65.04, 128.0, 70.05, 124.87, 108.86, 119.86, 102.6, 119.23, 86.95, 111.72, 86.95, 106.08, 107.61, 104.83, 107.61, 98.57, 90.71, 77.91, 86.33, 64.77, 86.33, 55.38, 90.08, 45.99, 100.73, 45.36, 109.49, 41.61, 110.12, 40.35, 97.6, 35.97, 89.46, 29.71, 86.95, 24.7, 93.84, 24.7, 105.73]], 'area': 5697.957150000001, 'iscrowd': 0, 'image_id': 26929, 'bbox': [8.43, 35.62, 127.08, 74.5], 'category_id': 14, 'id': 8876, 'dataset': 1}
{'segmentation': [[20.95, 105.11, 20.95, 87.58, 8.43, 80.69, 8.43, 75.06, 38.48, 74.43, 40.35, 66.92, 32.22, 63.79, 39.1, 35.62, 135.51, 37.5, 130.5, 65.04, 128.0, 70.05, 124.87, 108.86, 119.86, 102.6, 119.23, 86.95, 111.72, 86.95, 106.08, 107.61, 104.83, 107.61, 98.57, 90.71, 77.91, 86.33, 64.77, 86.33, 55.38, 90.08, 45.99, 100.73, 45.36, 109.49, 41.61, 110.12, 40.35, 97.6, 35.97, 89.46, 29.71, 86.95, 24.7, 93.84, 24.7

### 4. Save fixed json

In [ ]:
# fast
mlabjson['categories']=copy.copy(newjson['categories'])
mlabjson['annotations']=copy.copy(newjson['annotations'])

In [21]:
# save
print('\n >> SAVING...')
jsonfile=rdir+'mlab_fix_equal.json'
with open(jsonfile, 'w') as f:
    json.dump(newjson, f)
print("JSON SAVED : {} \n".format(jsonfile))


 >> SAVING...
JSON SAVED : D:/external_datasets/mlabequal.json 



# #DEBUG 'mlab_fix_equal.json'

In [2]:
mlabjson =  json.load(open(rdir+'mlab_fix_equal.json'))

In [3]:
for k in mlabjson:
    print(k, len(mlabjson[k]))

info 5
licenses 9
categories 1261
videos 1488
images 177936
tracks 8132
segment_info 0
annotations 1954769
datasets 2


In [4]:
# datasets name and id
dset_l=[]
dset_l_id=[]
for d in mlabjson['datasets']:
    dset_l.append(d['name'])
    dset_l_id.append(d['id'])
print(dset_l, dset_l_id)

['COCO', 'TAO'] [1, 2]


In [5]:
# categories name and id
cat_l=[]
cat_l_id=[]
cat_l_dset=[]
for c in mlabjson['categories']:
    cat_l.append(c['name'])
    cat_l_id.append(c['id'])
    cat_l_dset.append(dset_l[c['dataset']-1]) # dset_l index is same as id-1
#print(cat_l_id)

In [6]:
# images filepath and id
img_l=[]
img_l_id=[]
for c in mlabjson['images']:
    img_l.append(c['file_name'])
    img_l_id.append(c['id'])

In [14]:
# annotations category_id, image_id, bbox, and dataset
ann_catid=[]
ann_imgid=[]
ann_bbox=[]
ann_dset=[]
for an in tqdm(mlabjson['annotations']):
    ann_catid.append(an['category_id'])
    ann_imgid.append(an['image_id'])
    ann_bbox.append(an['bbox'])
    ann_dset.append(an['dataset'])
#NOTE: "TRY is NOT a good idea!!!: if something fails you should not use that annotation
print(len(ann_catid), len(ann_imgid), len(ann_bbox), len(ann_dset))

 44%|█████████████████████████████                                     | 860001/1954769 [00:00<00:01, 899572.23it/s]


KeyError: 'category_id'

In [15]:
%debug

> <ipython-input-14-cad5cb95cd71>(7)<module>()
      5 ann_dset=[]
      6 for an in tqdm(mlabjson['annotations']):
----> 7     ann_catid.append(an['category_id'])
      8     ann_imgid.append(an['image_id'])
      9     ann_bbox.append(an['bbox'])



ipdb>  an


{'image_id': 26942, 'id': 860002, 'caption': 'A bicycle replica with a clock as the front wheel.', 'dataset': 1}


ipdb>  quit


In [15]:
cocojson =  json.load(open(rdir+'COCO/2017/annotations/instances_val2017.json'))

In [16]:
# annotations category_id, image_id, bbox, and dataset
ann_catid=[]
ann_imgid=[]
ann_bbox=[]
for an in tqdm(cocojson['annotations']):
    ann_catid.append(an['category_id'])
    ann_imgid.append(an['image_id'])
    ann_bbox.append(an['bbox'])
#NOTE: "TRY is NOT a good idea!!!: if something fails you should not use that annotation
print(len(ann_catid), len(ann_imgid), len(ann_bbox))

100%|█████████████████████████████████████████████████████████████████████| 36781/36781 [00:00<00:00, 994197.98it/s]

36781 36781 36781


In [ ]:
%debug

In [12]:
cocojson['annotations'][0]

{'segmentation': [[267.03,
   243.78,
   314.59,
   154.05,
   357.84,
   136.76,
   374.05,
   104.32,
   410.81,
   110.81,
   429.19,
   131.35,
   420.54,
   165.95,
   451.89,
   209.19,
   464.86,
   240.54,
   480,
   253.51,
   484.32,
   263.24,
   496.22,
   271.89,
   484.32,
   278.38,
   438.92,
   257.84,
   401.08,
   216.76,
   370.81,
   247.03,
   414.05,
   277.3,
   433.51,
   304.32,
   443.24,
   323.78,
   400,
   362.7,
   376.22,
   375.68,
   400,
   418.92,
   394.59,
   424.32,
   337.3,
   382.16,
   337.3,
   371.35,
   388.11,
   327.03,
   341.62,
   301.08,
   311.35,
   276.22,
   304.86,
   263.24,
   294.05,
   249.19]],
 'num_keypoints': 8,
 'area': 28292.08625,
 'iscrowd': 0,
 'keypoints': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  325,
  160,
  2,
  398,
  177,
  2,
  0,
  0,
  0,
  437,
  238,
  2,
  0,
  0,
  0,
  477,
  270,
  2,
  287,
  255,
  1,
  339,
  267,
  2,
  0,
  0,
  0,
  423,
  314,
  2,
  0,
  0,
 